In [ ]:
# Notes
# 1 check out this link to see how that’s done in a really easy way with the yield statement! 
# 2 You can just use a for loop iterating over num samples.
# 3 You may want to make use of np.random.normal(size=).
# 4 You can use np.sum(axis=) along the last axis and compare it to 0. Remember your output should be at best of type ’integer’ not ’boolean’
# 5 Use np.expand dims(,-1) before you yield. The output shapes should be (seq len,1) and (1) for input and target respectiveley.
# 6 For example, we chose 80.000.
# 7 Shuffling, Batching.... As always.
# 8 You can use a Dense layer for each gate with a hidden size specified by units and a sigmoid activation.
# 9 To do so, check out the argument bias initializer= of the keras Dense layer impleme- nation and the tf.keras.initializers objects.
# 10 For the concatenation of the input and the hidden state to pass on to the forget gate, you can make use of tf.concat( (), axis=).
# 11 The appropiate size would be bacth size, cell units and you may want to use tf.zeros()
# 12 Consider that you have a binary classification task. How many units should your readout layer haven and what activation function should you use?
# 13 Do not forget to initialize the LSTM states with zeros each time you call your model and pass those on to your LSTM
# 14 You may want to use slicing, -1 is always the last irrespective of the size

In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

# 1.1


def integration_task(seq_len, num_samples):
    # for num_samples times yield random signal of size seq_len
    # and a target (sum of noise signal is > or < than 1)
    target = 0
    
    for i in num_samples:
        target = target + i
        ran = np.random.normal(size=seq_len)

        
    if np.sum(axis=target) > 0:
        tar_check = 1
        ran = np.expand_dims(ran,-1)
        yield ran, tar_check 
    else:
        yield 1


    
    
def my_integration_task():
    
    num_samples = 80000
    seq_len = 25
    
    yield integration_task(seq_len, num_samples)


In [14]:
# ToDo: split data into training and test data

dataset = tf.data.Dataset.from_generator(my_integration_task, output_signature = tf.TensorSpec(shape=[10,1],dtype=tf.dtypes.float32))

# training_data, testing_data = train_test_split(dataset, test_size=0.2, random_state=25)


def prepare_data(ds):
    return ds.shuffle(1000).batch(32).prefetch(20)
                                         
dataset = prepare_data(dataset)

In [5]:
# 2 - The network

class LSTM_Cell(): # hidden_state, cell_state
    
    def __init__(self, units, unit_forget_bias = False):
        
        self.units = units
        self.state_size = units
        
        self.dense1 = tf.keras.layers.Dense(units, kernel_regularizer=kernel_regularizer, use_bias=False, activation = tf.nn.sigmoid)
        self.dense2 = tf.keras.layers.Dense(units, kernel_regularizer=kernel_regularizer, use_bias=False, activation = tf.nn.tanh)
         
        self.bias = tf.Variable(tf.zeros(units), name="LSTM_Cell_biases")
        
        # for forget gate:
        if unit_forget_bias == True:
            bias +1
            
        weights = list()
        
        
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                      initializer='uniform',
                                      name='kernel')
        
        # point wise multiplication of layers:
        # to_add_cell = np.dot(self.dense1, self.dense2)
        
        
        return
    
    def call(self, x, states): # tf.concat( (), axis=)
        
        prev_output = states[0]
                
        to_output = np.dot(pre_output * self.dense1) +  self.bias
            
        # out = tf.nn.tanh(to_output)
        
        return to_output
    
    

    
class LSTM_Layer():
    
    def __init__(self, cell):
        
        cell = LSTM_Cell(32, False)
    
    
    def call(self,x,states):
        # input is shape [batch_size, seq_len, input_size]
        # output is shape [batch_size, seq_len, output_size]
        
        for i in states:
            
        
        output_size = states[0]
        
        to_out = [x[0], x[1], output_size]
        
        return 
    
    def zero_states(self, batch_size):
        
        size = batch_size[0]
        zero_state = np.zeros((size,),dtype='i,i')
        
        return zero_state
    
    
    

class LSTM_Model():
    
    def __init__():
        
        # put into Layer?
        read_in_layer = tf.keras.layers.Dense(128, activation = "sigmoid")
        output_layer = tf.keras.layers.Dense(10, activation = "tanh")

    
    def call(self, x):
        
        out = self.read_in_layer(x)
        out = self.output_layer(x)
        
        return out

In [ ]:
# 3 - Training